# Validation

In [1]:
import glob, os, time
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio import features

import matplotlib.pyplot as plt
import scipy
import sklearn
from sklearn.ensemble import RandomForestClassifier
#from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from pathlib import Path
from IPython.display import display

print(f'Numpy : {np.__version__}')
print(f'Pandas : {pd.__version__}')
print(f'GeoPandas : {gpd.__version__}')
print(f'Scipy: {scipy.__version__}')
print(f'Scikit-learn: {sklearn.__version__}')

Numpy : 1.19.2
Pandas : 1.1.5
GeoPandas : 0.8.1
Scipy: 1.5.2
Scikit-learn: 0.24.1


## Set paths for input and output directories

In [5]:
#computer_path = 'X:/'
computer_path = '/Volumes/nbdid-sst-lbrat2104/'
grp_letter    = 'A'

# Directory for all work files
#work_path = f'{computer_path}GROUP_{grp_letter}/WORK/'

work_path = f'{computer_path}GROUP_{grp_letter}/TP/llesne/DATA/'


# ----- #
# INPUT #
# ----- #

im_path   = f'{work_path}3_L2A_MASKED/'
ndvi_path = f'{work_path}NDVI/'

in_situ_SD_path = f'{work_path}IN_SITU_SD/'

# ------ #
# OUTPUT #
# ------ #

classif_path = f'{work_path}CLASSIF/'

Path(classif_path).mkdir(parents=True, exist_ok=True)

print(f'Classification path are set to : {classif_path}')

Classification path are set to : /Volumes/nbdid-sst-lbrat2104/GROUP_A/TP/llesne/DATA/CLASSIF/


## Rasterize in-situ data shapefile

In [8]:
# Set the field name of the class code

field_name_code = 'CLASS_NIV3'

# Set up filenames

in_situ_name = 'COSW_IN_SITU_ROI'

#shp    = f'{in_situ_SD_path}{in_situ_name}_cal.shp'
#raster = f'{in_situ_SD_path}{in_situ_name}_cal.tif'

shp    = f'{in_situ_SD_path}{in_situ_name}_val.shp'
raster = f'{in_situ_SD_path}{in_situ_name}_val.tif'


print(f'{ndvi_path}*.tif')

img_temp_tif = glob.glob(f'{ndvi_path}*.tif')[0]

print(f'Raster template file : {img_temp_tif}')

print(f'Rasterize {shp}')

# Open the shapefile with GeoPandas

in_situ_gdf = gpd.read_file(shp).to_crs(epsg='32631')

print(in_situ_gdf.crs)

# Open the raster file you want to use as a template for rasterize

src = rasterio.open(img_temp_tif, "r")

# Update metadata
profile = src.profile
profile.update(nodata=0)

# Burn the features into the raster and write it out

dst = rasterio.open(raster, 'w+', **profile)
dst_arr = dst.read(1)

# this is where we create a generator of geom, value pairs to use in rasterizing

geom_col = in_situ_gdf.geometry
code_col = in_situ_gdf[field_name_code].astype(int)

shapes = ((geom,value) for geom, value in zip(geom_col, code_col))

in_situ_arr = features.rasterize(shapes=shapes,
                                    fill=0,
                                    out=dst_arr,
                                    transform=dst.transform)

dst.write_band(1, in_situ_arr)

# Close rasterio objects
src.close()
dst.close()


/Volumes/nbdid-sst-lbrat2104/GROUP_A/TP/llesne/DATA/NDVI/*.tif
Raster template file : /Volumes/nbdid-sst-lbrat2104/GROUP_A/TP/llesne/DATA/NDVI/NDVI_1.tif
Rasterize /Volumes/nbdid-sst-lbrat2104/GROUP_A/TP/llesne/DATA/IN_SITU_SD/COSW_IN_SITU_ROI_val.shp
epsg:32631
